https://pythonlib.ru/library-theme9

# Введение

**CatBoost (Categorical Boosting)** - современный алгоритм градиентного бустинга, созданный командой разработчиков Яндекса. Эта библиотека машинного обучения специально оптимизирована для работы с категориальными признаками и предназначена для решения широкого спектра задач: от классификации и регрессии до ранжирования и рекомендательных систем.

**Ключевые преимущества CatBoost:**

*Автоматическая обработка категориальных признаков.*
CatBoost революционизирует подход к работе с категориальными данными. 
Традиционные алгоритмы требуют предварительного преобразования категориальных переменных через one-hot encoding или label encoding. 
CatBoost же использует собственный метод статистической агрегации по истории (CTR - Click-Through Rate), который автоматически обрабатывает категориальные признаки без потери информации.

*Высокая точность и устойчивость к переобучению.*
Алгоритм демонстрирует отличные результаты на различных типах данных благодаря использованию симметричных деревьев и продвинутых методов регуляризации. Встроенные механизмы предотвращения переобучения позволяют получать стабильные результаты даже на небольших выборках.

*Гибкость в вычислительных ресурсах.*
CatBoost поддерживает как CPU-, так и GPU-обучение, что позволяет значительно ускорить процесс обучения на больших объемах данных. Библиотека эффективно работает как с небольшими наборами данных, так и с большими промышленными датасетами.

*Интеграция с популярными инструментами.*
Полная совместимость с экосистемой Python для анализа данных: Pandas, NumPy, Scikit-learn. Это обеспечивает легкую интеграцию в существующие пайплайны машинного обучения.

*Встроенные инструменты анализа.*
CatBoost предоставляет богатые возможности для визуализации процесса обучения, анализа важности признаков и мониторинга качества модели в реальном времени.


**Технические особенности алгоритма:**

*Обработка категориальных признаков.*
Главное отличие CatBoost от других алгоритмов бустинга заключается в способе обработки категориальных переменных. Вместо традиционных методов кодирования, CatBoost использует target-based статистики, вычисляемые на основе исторических данных. Это позволяет сохранить всю информацию о категориальных признаках без увеличения размерности данных.

*Симметричные деревья.*
CatBoost строит симметричные (сбалансированные) деревья, что обеспечивает лучшую обобщающую способность и стабильность предсказаний. Это отличается от подхода XGBoost и LightGBM, которые используют листовые деревья.

*Обработка пропущенных значений.*
Алгоритм автоматически обрабатывает пропущенные значения без необходимости дополнительной предобработки данных. CatBoost рассматривает пропуски как отдельную категорию и эффективно их обрабатывает.

*Поддержка текстовых признаков.*
CatBoost может работать с текстовыми данными, автоматически извлекая признаки из текста и используя их для обучения модели.

**Особенности:**

- CatBoost НЕ принимает NaN в категориальных признаках, но принимает в числовых

In [ ]:
# !pip install catboost
# !pip install catboost[gpu]

In [ ]:
from catboost import CatBoostClassifier, CatBoostRegressor, CatBoostRanker
from catboost import Pool, cv, sum_models

# Подготовка данных 

In [1]:
from catboost.datasets import titanic

titanic_train, titanic_val = titanic()

In [2]:
titanic_train.head(1)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


In [3]:
titanic_val.head(1)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q


## Работа с категориальными признаками
CatBoost может автоматически определять категориальные признаки, но для лучшего контроля рекомендуется указывать их явно:

In [39]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier

# Загрузка данных
df = titanic_train

# Определение категориальных признаков
cat_features = ['Sex', 'Embarked', 'Cabin']

for col in cat_features:
    df[col] = df[col].fillna('Missing')

# Подготовка данных
y_feature = 'Survived'
X_drop_features = ['Name', 'Ticket']
X_drop_features_plus_y = X_drop_features + [y_feature]
X = df.drop(X_drop_features_plus_y, axis=1)
y = df[y_feature]

In [5]:
X.head(1)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,3,male,22.0,1,0,7.25,Missing,S


## Использование Pool для оптимизации !
Pool - это специальный объект CatBoost для хранения данных, который обеспечивает более эффективную работу с большими датасетами: 

In [ ]:
# from catboost import Pool

# # Создание Pool объекта
# train_pool = Pool(
#     data=X_train,
#     label=y_train,
#     cat_features=cat_features,
#     feature_names=list(X_train.columns)
# )

# eval_pool = Pool(
#     data=X_eval,
#     label=y_eval,
#     cat_features=cat_features,
#     feature_names=list(X_eval.columns)
# )

# Обучение моделей классификации
Базовый пример

In [9]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

### Подберем оптимальные параметры на кросс-валидации

Автоматически подбирает модель (классикация / регрессия) по параметру loss_function.

Служит для быстрого подбора гиперпараметров на кросс-валидации

In [81]:
from catboost import cv
from catboost import Pool
from sklearn.model_selection import cross_val_score

# Использование встроенной кросс-валидации CatBoost
cv_results = cv(
    pool=Pool(X_train, y_train, cat_features=cat_features),
    params={
        'iterations': 1000,
        'learning_rate': 0.01,
        # 'depth': 4,
        'l2_leaf_reg': 3,
        # 'bagging_temperature': 5,
        # 'random_strength': 4,
        'loss_function': 'Logloss',
        'custom_loss': ['Accuracy'],
    },
    fold_count=3,
    shuffle=True,
    stratified=True, # для равномерного распределения классов
    verbose=100,
    plot=True,
    partition_random_seed=1
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/3]
0:	learn: 0.6889785	test: 0.6891840	best: 0.6891840 (0)	total: 7.6ms	remaining: 15.2s
100:	learn: 0.4241770	test: 0.4801685	best: 0.4801685 (100)	total: 763ms	remaining: 14.3s
200:	learn: 0.3612339	test: 0.4541406	best: 0.4541406 (200)	total: 1.5s	remaining: 13.5s
300:	learn: 0.3341580	test: 0.4497541	best: 0.4497036 (282)	total: 2.26s	remaining: 12.7s
400:	learn: 0.3094725	test: 0.4454188	best: 0.4451688 (394)	total: 3.01s	remaining: 12s
500:	learn: 0.2891642	test: 0.4439654	best: 0.4433805 (491)	total: 3.76s	remaining: 11.3s
600:	learn: 0.2712949	test: 0.4422688	best: 0.4422688 (600)	total: 4.59s	remaining: 10.7s
700:	learn: 0.2550843	test: 0.4430986	best: 0.4416161 (644)	total: 5.37s	remaining: 9.95s
800:	learn: 0.2379295	test: 0.4408794	best: 0.4398137 (779)	total: 6.18s	remaining: 9.25s
900:	learn: 0.2229342	test: 0.4400295	best: 0.4398137 (779)	total: 7.01s	remaining: 8.55s
1000:	learn: 0.2067240	test: 0.4391862	best: 0.4390930 (971)	total: 7.82s	remaining:

KeyboardInterrupt: 

In [79]:
cv_results.head(3)

,iterations,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std,test-Accuracy-mean,test-Accuracy-std,train-Accuracy-mean,train-Accuracy-std
0,0,0.687420,0.001557,0.687533,0.001272,0.800594,0.020274,0.798453,0.004991
1,1,0.682824,0.001327,0.682351,0.000286,0.786542,0.021244,0.792143,0.012497
2,2,0.678033,0.001204,0.676588,0.001882,0.785135,0.018914,0.807605,0.025957


In [80]:
best_value = np.max(cv_results['test-Accuracy-mean'])
best_iter = np.argmax(cv_results['test-Accuracy-mean'])

print(best_value)
print(best_iter)

0.8188313300003546
933


### Обучение

In [82]:
# Создание и обучение модели
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.01,
    # depth=4,
    loss_function='Logloss',
    eval_metric='Accuracy',
    random_seed=42,
    verbose=100,
    custom_loss=['Accuracy'],
)

In [83]:
print(model.is_fitted())
print(model.get_params())

False
{'iterations': 1000, 'learning_rate': 0.01, 'loss_function': 'Logloss', 'random_seed': 42, 'verbose': 100, 'custom_loss': ['Accuracy'], 'eval_metric': 'Accuracy'}


In [84]:
# Обучение с валидацией
model.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_test, y_test),
    early_stopping_rounds=1000,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.8272472	test: 0.7653631	best: 0.7653631 (0)	total: 17.1ms	remaining: 17.1s
100:	learn: 0.8342697	test: 0.8044693	best: 0.8100559 (12)	total: 823ms	remaining: 7.33s
200:	learn: 0.8539326	test: 0.7988827	best: 0.8100559 (12)	total: 1.69s	remaining: 6.71s
300:	learn: 0.8679775	test: 0.7988827	best: 0.8100559 (12)	total: 2.59s	remaining: 6.03s
400:	learn: 0.8679775	test: 0.8100559	best: 0.8156425 (393)	total: 3.49s	remaining: 5.22s
500:	learn: 0.8806180	test: 0.7988827	best: 0.8156425 (393)	total: 4.33s	remaining: 4.31s
600:	learn: 0.8890449	test: 0.8100559	best: 0.8156425 (393)	total: 5.14s	remaining: 3.41s
700:	learn: 0.8932584	test: 0.8100559	best: 0.8156425 (393)	total: 6.04s	remaining: 2.58s
800:	learn: 0.8960674	test: 0.8100559	best: 0.8156425 (393)	total: 6.95s	remaining: 1.73s
900:	learn: 0.9030899	test: 0.8044693	best: 0.8156425 (393)	total: 7.84s	remaining: 862ms
999:	learn: 0.9143258	test: 0.7988827	best: 0.8156425 (393)	total: 8.74s	remaining: 0us

bestTest = 0.8156

In [85]:
# Предсказание
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

In [86]:
# Оценка качества
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred))

Accuracy: 0.8156
              precision    recall  f1-score   support

           0       0.81      0.91      0.86       110
           1       0.82      0.67      0.74        69

    accuracy                           0.82       179
   macro avg       0.82      0.79      0.80       179
weighted avg       0.82      0.82      0.81       179



## Обучение на ВСЕМ датасете (train + test)

In [87]:
tree_count = model.tree_count_

In [88]:
tree_count

394

In [89]:
model = CatBoostClassifier(
    iterations=int(model.tree_count_ * 1.2),
    learning_rate=0.01,
    depth=4,
    loss_function='Logloss',
    eval_metric='Accuracy',
    random_seed=42,
    verbose=100,
    custom_loss=['Accuracy'],
)

model.fit(
    X, y,
    cat_features=cat_features,
    # early_stopping_rounds=1000,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.7901235	total: 13.5ms	remaining: 6.36s
100:	learn: 0.8114478	total: 619ms	remaining: 2.27s
200:	learn: 0.8193042	total: 1.26s	remaining: 1.71s
300:	learn: 0.8395062	total: 1.86s	remaining: 1.06s
400:	learn: 0.8428732	total: 2.47s	remaining: 438ms
471:	learn: 0.8428732	total: 2.9s	remaining: 0us


In [90]:
model.tree_count_

472

## validation

In [91]:
# Загрузка данных
df_val = titanic_val

for col in cat_features:
    df_val[col] = df_val[col].fillna('Missing')

X_val = df_val.drop(X_drop_features, axis=1)

y_pred_val = model.predict(X_val)

df_val_finish = pd.concat([df_val['PassengerId'], pd.Series(y_pred_val, name='Survived')], axis=1)

df_val_finish = df_val_finish.set_index('PassengerId')

df_val_finish.to_csv('tit.csv')

# Настройка гиперпараметров

In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV

# Определение сетки параметров
param_grid = {
    # 'iterations': [500, 1000, 1500],
    'learning_rate': [0.01, 0.1],
    'depth': [4, 6],
    'l2_leaf_reg': [3, 5]
}

# Создание модели
model = CatBoostClassifier(
    random_seed=42,
    verbose=0,
    cat_features=cat_features
)

# Поиск по сетке
grid_search = GridSearchCV(
    model, param_grid, 
    cv=3, scoring='roc_auc',
    n_jobs=-1, verbose=1
)

grid_search.fit(X_train, y_train)
print(f"Лучшие параметры: {grid_search.best_params_}")
print(f"Лучший результат: {grid_search.best_score_:.4f}")

In [ ]:
# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Создание и обучение модели
model = CatBoostClassifier(
    # iterations=1000,
    learning_rate=0.01,
    depth=4,
    l2_leaf_reg=3,
    loss_function='Logloss',
    eval_metric='Accuracy',
    random_seed=42,
    verbose=100
)

# Обучение с валидацией
model.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_test, y_test),
    # early_stopping_rounds=50,
    plot=True
)

In [ ]:
# Загрузка данных
df_val = titanic_val

for col in cat_features:
    df_val[col] = df_val[col].fillna('Missing')

X_val = df_val.drop(X_drop_features, axis=1)

y_pred_val = model.predict(X_val)

df_val_finish = pd.concat([df_val['PassengerId'], pd.Series(y_pred_val, name='Survived')], axis=1)

df_val_finish = df_val_finish.set_index('PassengerId')

df_val_finish.to_csv('tit.csv')

# Обучение моделей регрессии

In [ ]:
from catboost.datasets import monotonic2
monotonic2_train, monotonic2_test = monotonic2()

In [ ]:
monotonic2_train.head(3)

In [ ]:
monotonic2_train.describe()

In [ ]:
X_train = monotonic2_train.drop(['Target'], axis=1)
X_test = monotonic2_test.drop(['Target'], axis=1)

In [ ]:
y_train = monotonic2_train['Target']
y_test = monotonic2_test['Target']

In [ ]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error

cat_features = []

# Создание модели регрессии
regressor = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=8,
    loss_function='RMSE',
    eval_metric='MAE',
    random_seed=42,
    verbose=100
)

# Обучение
regressor.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_test, y_test),
    early_stopping_rounds=100
)

In [ ]:
# Предсказание
y_pred = regressor.predict(X_test)

# Оценка
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae:.4f}")
r2_score_ = r2_score(y_test, y_pred)
print(f"r2_score_: {r2_score_:.4f}")
root_mean_squared_error_ = root_mean_squared_error(y_test, y_pred)
print(f"root_mean_squared_error_: {root_mean_squared_error_:.4f}")

In [ ]:
from catboost import cv
from catboost import Pool
from sklearn.model_selection import cross_val_score

# Использование встроенной кросс-валидации CatBoost
cv_results = cv(
    pool=Pool(X_train, y_train, cat_features=[]),
    params={
        'iterations': 1000,
        'learning_rate': 0.1,
        'depth': 6,
        'loss_function': 'RMSE'
    },
    fold_count=5,
    shuffle=True,
    stratified=True, # для равномерного распределения классов
    seed=42,
    verbose=100,
    plot=True
)

# cv_results
print(f"Среднее значение RMSE: {cv_results['test-RMSE-mean'].iloc[-1]:.4f}")

In [ ]:
cv_results